### The Reuters dataset

We'll classify Routers news into 46 categories

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import reuters
from tensorflow.keras import models, layers

# load datasets that comes with Keras
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

In [3]:
len(train_data)

8982

In [4]:
len(test_data)

2246

### Vectorize dataset

In [5]:
# Vectorize dataset
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.0
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

### Vectorize labels

Since our labels is multiclass:
- We can cast the labels into integer tensor
- We can one-hot encode them (categorical encoding)

Here we'll to one-hot encoding using Keras utils.

In [6]:
# Vectorize our label
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

In [7]:
y_train.shape

(8982, 46)

In [8]:
y_train[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

### Build model

In [12]:
def get_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(46, activation='softmax')) # layer's node count = 46, need to match the class/category length
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
    return model

In [13]:
# Split training dataset into training and validation
x_val = x_train[:1000]
x_train2 = x_train[1000:]

y_val = y_train[:1000]
y_train2 = y_train[1000:]

### Run training

In [14]:
model = get_model()

EPOCHS = 20
history = model.fit(x_train2, y_train2, epochs=EPOCHS, batch_size=512, validation_data=(x_val, y_val))

Epoch 1/20
16/16 [==============================] - 0s 24ms/step - loss: 2.6365 - acc: 0.5113 - val_loss: 1.7934 - val_acc: 0.6480
Epoch 2/20
16/16 [==============================] - 0s 15ms/step - loss: 1.4385 - acc: 0.7052 - val_loss: 1.3261 - val_acc: 0.7150
Epoch 3/20
16/16 [==============================] - 0s 15ms/step - loss: 1.0586 - acc: 0.7746 - val_loss: 1.1461 - val_acc: 0.7540
Epoch 4/20
16/16 [==============================] - 0s 15ms/step - loss: 0.8299 - acc: 0.8222 - val_loss: 1.0592 - val_acc: 0.7640
Epoch 5/20
16/16 [==============================] - 0s 15ms/step - loss: 0.6668 - acc: 0.8599 - val_loss: 0.9799 - val_acc: 0.8060
Epoch 6/20
16/16 [==============================] - 0s 14ms/step - loss: 0.5301 - acc: 0.8929 - val_loss: 0.9280 - val_acc: 0.8150
Epoch 7/20
16/16 [==============================] - 0s 15ms/step - loss: 0.4316 - acc: 0.9127 - val_loss: 0.9097 - val_acc: 0.8060
Epoch 8/20
16/16 [==============================] - 0s 14ms/step - loss: 0.3486 - a

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, EPOCHS + 1)

plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training & Validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

train_acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(epochs, train_acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training & Validation acc')
plt.xlabel('epochs')
plt.ylabel('acc')
plt.legend()
plt.show()

#### Our model overfit after 9 epochs, lets train using all data with 9 epochs

In [ ]:
model = get_model()
model.fit(x_train, y_train, epochs=9, batch_size=512)

In [ ]:
results = model.evaluate(x_test, y_test)

In [ ]:
print("Model Accuracy: {}%".format(results[1]))

In [ ]:
# Make prediction

predictions = model.predict(x_test)

In [ ]:
predictions[0]

In [ ]:
predictions[0].shape

In [ ]:
np.sum(predictions[0])

In [ ]:
np.argmax(predictions[0])

In [ ]:
train_labels

In [ ]:
np.array(train_labels)

### Another way to handle multicategorical labels

Is to cast them as integer tensor and use different loss function called `sparse_categorical_crossentropy`

In [ ]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [ ]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.fit(x_train, y_train, epochs=9, batch_size=512, validation_data=())

In [ ]:
results = model.evaluate(x_test, y_test)

In [ ]:
print("Model Accuracy: {}%".format(results[1]))

In [ ]:
# Similar result to when label are one-hot encoded.